# 逆行列


$n$次正方行列$A$に対し

$$
A^{-1}A = AA^{-1} = I
$$

を満たす正方行列$A^{-1}$が存在するとき、$A$は **正則行列（regular matrix）** や **可逆行列（invertible matrix）** とよばれる。

また、$A^{-1}$を$A$の**逆行列**という。（$I$は$A$と同じサイズの単位行列）

正則でない行列は **特異行列（singular matrix）** という。

### 正則行列の条件

正方行列$A$が正則行列であるための必要十分条件は$\det(A) \neq 0$である。

$A$の逆行列$A^{-1}$の計算方法の一つに

$$
A^{-1} = \frac{1}{ \det(A) } \tilde{A}
$$

があり、分母が$\det(A) \neq 0$である必要がある。ここで$\tilde{A}$は$A$の余因子行列である。

In [1]:
import numpy as np

A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0],
])

def cofactor(A, i, j):
    """余因子を計算する関数"""
    A_minor = np.delete(A, i, axis=0)
    A_minor = np.delete(A_minor, j, axis=1)
    return (-1)**(i + j) * np.linalg.det(A_minor)

def cofactor_matrix(A):
    """余因子行列"""
    C = np.zeros_like(A, dtype=np.float32)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            C[i, j] = cofactor(A, i=i, j=j)
    return C.T

# det(A) != 0 かどうかを確認
print(f"det(A) = {np.linalg.det(A)}")

# 余因子行列の計算
A_adj = cofactor_matrix(A)
print(f"\n余因子行列 A_adj:\n{A_adj}")

# 逆行列
A_inv = (1 / np.linalg.det(A)) * A_adj
print(f"\n逆行列 A^-1:\n{A_inv.round(3)}")

det(A) = 27.0

余因子行列 A_adj:
[[-48.  24.  -3.]
 [ 42. -21.   6.]
 [ -3.   6.  -3.]]

逆行列 A^-1:
[[-1.778  0.889 -0.111]
 [ 1.556 -0.778  0.222]
 [-0.111  0.222 -0.111]]


In [18]:
# ちゃんと逆行列になっているか確認
(A_inv @ A).round(1)

array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [ 0.,  0.,  1.]])

### 余因子行列と逆行列

$A$を$n$次正方行列、$\tilde{A}$を$A$の余因子行列、$I$を$n$次単位行列とすると

$$
A \tilde{A}
= \tilde{A} A
= \det(A) I
$$

という関係がある。

なので、$A$が正則であれば

$$
\underbrace{ \frac{1}{\det(A)} \tilde{A} }_{A^{-1}} A = I
$$

となる。

In [24]:
print(f"det(A) = {np.linalg.det(A):.0f}")
print(f"A_adj @ A: \n{A_adj @ A}")

det(A) = 27
A_adj @ A: 
[[27.  0.  0.]
 [ 0. 27.  0.]
 [ 0.  0. 27.]]


## 逆行列の計算規則

:::{admonition} 定理

$$
(A^{-1})^{-1} = A
$$
:::

:::{dropdown} 証明

逆行列の定義より $(A^{-1})^{-1}$ついて

$$
(A^{-1}) (A^{-1})^{-1}
= (A^{-1})^{-1} (A^{-1})
= I
$$

が成り立つことになる。これを満たすのは $(A^{-1})^{-1} = A$である
:::

:::{admonition} 定理

$$
(AB)^{-1} = B^{-1} A^{-1}
$$
:::

:::{dropdown} 証明

$B^{-1} A^{-1}$に右から$AB$を掛けると

$$
B^{-1} \underbrace{ A^{-1} A}_{I} B
= B^{-1} B
= I
$$

左から掛けると

$$
A B B^{-1} A^{-1}
= A A^{-1}
= I
$$

となる。よって

$$
(AB) (B^{-1} A^{-1}) = (B^{-1} A^{-1}) (AB) = I
$$

となるため、逆行列の定義より

$$
B^{-1} A^{-1} = (AB)^{-1}
$$

である。
:::

:::{admonition} 定理

$$
(A^k)^{-1} = (A^{-1})^k
$$
:::


:::{admonition} 定理

$$
(A^\top)^{-1} = (A^{-1})^\top
$$
:::

:::{dropdown} 証明

$$
A A^{-1} = A^{-1} A = I
$$

の両辺の転置行列を考えると

$$
(A A^{-1})^\top = (A^{-1} A)^\top = I^\top
$$

転置の性質$(AB)^\top = B^\top A^\top$と$I^\top = I$より

$$
(A^{-1})^\top A^\top
= A^\top (A^{-1})^\top
= I
$$

となる。よって$A^\top$は正則であり、その逆行列は$(A^{-1})^\top$である。
:::

In [2]:
import numpy as np

A = np.array([
    [1, 2],
    [3, 4],
])
B = A + 3
A_inv = np.linalg.inv(A)

A_inv @ B @ A

array([[-11., -16.],
       [ 15.,  22.]])